In [1]:
%run 'Classifier.ipynb'

<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>


In [2]:
# Import data from spreadsheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("Habits to achieve your goals").sheet1

# Extract the values
list_of_habits = sheet.col_values(2)[1:]
list_of_goals = sheet.col_values(3)[1:]
session_length = sheet.col_values(4)[1:]
frequency = sheet.col_values(5)[1:]
time_to_result = sheet.col_values(7)[1:]
categories = sheet.col_values(8)[1:]

In [3]:
# Data processing - convert choices into numerical score
session_length_score = []
for i in session_length:
    if i == 'Under 3 minutes':
        session_length_score.append(1)
    if i == '3 to under 15 minutes':
        session_length_score.append(2)
    if i == '15 to under 30 minutes':
        session_length_score.append(3)
    if i == '30 to under 60 minutes':
        session_length_score.append(4)
    if i == '1 to under 2 hours':
        session_length_score.append(5)
    if i == '2 to under 4 hours':
        session_length_score.append(6)
    if i == '4 hours or more':
        session_length_score.append(7)

frequency_score = []
for i in frequency:
    if i == 'Once a year or less':
        frequency_score.append(1)
    if i == '2 to 4 times a year':
        frequency_score.append(2)
    if i == '5 to 10 times a year':
        frequency_score.append(3)
    if i == 'Roughly once a month':
        frequency_score.append(4)
    if i == '2 to 3 times a month':
        frequency_score.append(5)
    if i == 'Roughly once a week':
        frequency_score.append(6)
    if i == '2 to 5 times a week':
        frequency_score.append(7)
    if i == 'Every day (or almost)':
        frequency_score.append(8)
    if i == '8 to 12 times a week':
        frequency_score.append(9)
    if i == 'Twice a day or more':
        frequency_score.append(10)
        
time_to_result_score = []
for i in time_to_result:
    if i == 'Less than one week':
        time_to_result_score.append(1)
    if i == '1 to under 3 weeks':
        time_to_result_score.append(2)
    if i == '3 to under 8 weeks':
        time_to_result_score.append(3)
    if i == '2 to under 4 months':
        time_to_result_score.append(4)
    if i == '4 to under 6 months':
        time_to_result_score.append(5)
    if i == '6 to under 12 months':
        time_to_result_score.append(6)
    if i == '1 to under 2 years':
        time_to_result_score.append(7)
    if i == '2 to 4 years':
        time_to_result_score.append(8)
    if i == 'More than 4 years':
        time_to_result_score.append(9)

In [4]:
import itertools, nltk, string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):
    
    # Exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # Tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # Join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O') if key]

    return [cand for cand in candidates
            if cand not in stop_words and not all(char in punct for char in cand)]

In [5]:
from autocorrect import spell

def spelling(word):
    # Do not spell-check acronyms
    if word == word.upper():
        return word
    else:
        return spell(word)

def correct(word):
    # Split a phrase into seperate words, spell-check them
    # and join them back
    word = word.split(" ")
    word = [spelling(i) for i in word]
    word = " ".join(word)
    return word

corrected_habits = map(lambda x: correct(x), list_of_habits)
keywords_habits = []
for i in corrected_habits:
    if len(i.split(" ")) > 2:
        keywords_habits.append(extract_candidate_chunks(i))
    else:
        keywords_habits.append(i)
            
categorized_goals = pipeline.predict(list_of_goals)

In [12]:
def find_habit():
    goal = raw_input("Enter your goals: ")
    print "How long each session would you like it to be?"
    length = raw_input("Type 1 for under an hour, 2 for an hour or more: ")
    print "How often would you like to engage in this activity?"
    print "Type 1 for once a week or more frequently,"
    frequent = raw_input("or 2 for 2 to 3 times a month or less frequently: ")
    print "How long do you what it to start seeing results?"
    time = raw_input("Type 1 for under 2 months, or 2 for 2 months or more: ")
    print ''
    category = pipeline.predict([goal])
    goals = []
    habits = []
    for i in range(len(categorized_goals)):
        if category == categorized_goals[i] and list_of_goals[i] != '':
            goals.append([categorized_goals[i], i])
            habits.append(keywords_habits[i])

    goals_1 = []
    habits_1 = []
    if length == "1":
        for i in range(len(goals)):
            if session_length_score[goals[i][1]] < 5:
                goals_1.append(goals[i])
                habits_1.append(keywords_habits[goals[i][1]])
    else:
        for i in range(len(goals)):
            if session_length_score[goals[i][1]] >= 5:
                goals_1.append(goals[i])
                habits_1.append(keywords_habits[goals[i][1]])

    goals_2 = []
    habits_2 = []
    if frequent == "1":
        for i in range(len(goals_1)):
            if frequency_score[goals_1[i][1]] > 5:
                goals_2.append(goals[i])
                habits_2.append(keywords_habits[goals[i][1]])
    else:
        for i in range(len(goals_1)):
            if frequency_score[goals_1[i][1]] <= 5:
                goals_2.append(goals[i])
                habits_2.append(keywords_habits[goals[i][1]])
    goals_3 = []
    habits_3 = []
    if time == "1":
        for i in range(len(goals_2)):
            if time_to_result_score[goals_2[i][1]] <= 3:
                goals_3.append(goals[i])
                habits_3.append(keywords_habits[goals[i][1]])
    else:
        for i in range(len(goals_2)):
            if time_to_result_score[goals_2[i][1]] > 3:
                goals_3.append(goals[i])
                habits_3.append(keywords_habits[goals[i][1]])
    if habits_3 != []:
        print "Here are the habits matching your choice:"
        print habits_3
    elif habits_2 != []:
        print "We do not have any habits matching your choice."
        print "However, here are some close matches:"
        print habits_2
    elif habits_1 != []:
        print "We do not have any habits matching your choice."
        print "However, here are some close matches:"
        print habits_1
    elif habits != []:
        print "We do not have any habits matching your choice."
        print "However, here are some close matches:"
        print habits
    else:
        print "Sorry, we do not have any close matches to your choice."
        
simulation_flag = True
while simulation_flag:
    find_habit()
    check = raw_input("Do you want to try again? (y/n): ")
    if check == 'n':
        simulation_flag = False 

Enter your goals: get six-pack
How long each session would you like it to be?
Type 1 for under an hour, 2 for an hour or more: 1
How often would you like to engage in this activity?
Type 1 for once a week or more frequently,
or 2 for 2 to 3 times a month or less frequently: 1
How long do you what it to start seeing results?
Type 1 for under 2 months, or 2 for 2 months or more: 2

Here are the habits matching your choice:
[['meditation', 'past'], ['tv shows'], 'Weight lifting', 'Reading medicine', ['track', 'daily personal swot analyses'], 'Journaling', ['gym', 'mma', 'football'], ['coffee', 'music', 'morning go', 'gym', 'other day'], 'Reading philosophy', [u'yoga', u'signing'], 'running', 'Weightlifting', ['gym', '2-3 times', 'week'], ['gym'], ['glass of water', 'coffee go', 'gym', 'prepare', 'clothes', 'gym', 'night', 'day', 'week', 'sundays', 'day for day', 'clothes', 'calender etc go', 'bed', 'pm', 'night'], ['gym', 'sit', 'phone', 'time', 'music', 'outside world'], ['attend footbal